In [24]:
import pandas as pd
phenotype_to_genes = pd.read_csv('/content/phenotype_to_genes.txt', sep='\t')
genes_to_phenotype = pd.read_csv('/content/genes_to_phenotype.txt', sep='\t')
genes_to_disease = pd.read_csv('/content/genes_to_disease.txt', sep='\t')
phenotype = pd.read_csv('/content/phenotype.hpoa', sep='\t', skiprows=4)


<ipython-input-24-dc7a4f8dc0e6>:5: DtypeWarning: Columns (2,6,8,9) have mixed types. Specify dtype option on import or set low_memory=False.
  phenotype = pd.read_csv('/content/phenotype.hpoa', sep='\t', skiprows=4)


In [25]:
phenotype=phenotype[['database_id',	'disease_name']].drop_duplicates()
phenotype_dic=dict()
for i in phenotype.index:
    phenotype_dic[phenotype['database_id'][i]]   =  phenotype['disease_name'][i]

def transform(x, y,z):
     try:
        y=phenotype_dic[y]
        return         f"<s>[INST] What disease is associated with the gene {x}? [/INST] The gene {x} is associated with the disease {y} based on a {z} relationship. </s>"
     except:
        return 'None'
genes_to_disease['text']=genes_to_disease.apply(lambda x: transform(x['gene_symbol'], x['disease_id'],x['association_type']),axis=1)
genes_to_disease=genes_to_disease[genes_to_disease['text'] != 'None']

In [20]:
import pandas as pd
data_=pd.read_csv('/content/FT_data_v2.csv')
def transform(x, y, z):
    if z == 'both description':
        return f"<s>[INST] What does this concept mean? {x} [/INST] {y} </s>"
    elif z == 'description':
        return f"<s>[INST] Describe {x}: [/INST] {x} is a rare disease. {y} </s>"
    elif z == 'synonym':
        return f"<s>[INST] What is the synonym of {x}? [/INST] {y} is a synonym of {x}. </s>"
    else:
        return f"<s>[INST] Explain {x}: [/INST] {y} </s>"  # Default case

data_['text']= data_.apply(lambda x: transform(x['sentence1'], x['sentence2'], x['similarity_score']),axis=1)


In [16]:
df = phenotype_to_genes[['hpo_name','gene_symbol']]
df['gene_symbol'] = df.groupby('hpo_name')['gene_symbol'].transform(lambda x: ', '.join(x))
df = df.drop_duplicates(subset=['hpo_name'])
df=pd.DataFrame(df)
def transform(x, y):
        return         f"<s>[INST] What genes are associated with the phenotype {x}? [/INST] The phenotype {x} is associated with the follwoing genes: {y} . </s>"

df['text']=df.apply(lambda x: transform(x['hpo_name'],x['gene_symbol'] ),axis=1)

<ipython-input-16-70d61bf2dab1>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['gene_symbol'] = df.groupby('hpo_name')['gene_symbol'].transform(lambda x: ', '.join(x))


In [17]:
df2 = genes_to_phenotype[['gene_symbol', 'hpo_name']]
df2['hpo_name'] = df2.groupby('gene_symbol')   ['hpo_name'].transform(lambda x: ', '.join(x))
df2 = df2.drop_duplicates(subset=['gene_symbol'])
df2=pd.DataFrame(df2)
df2
def transform(x, y):
        return         f"<s>[INST] What phenotypes are associated with the gene {x}? [/INST] The gene {x} is associated with the follwoing phenotypes: {y} . </s>"

df2['text']=df2.apply(lambda x: transform(x['gene_symbol'], x['hpo_name']),axis=1)

<ipython-input-17-dc29b2963cad>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['hpo_name'] = df2.groupby('gene_symbol')   ['hpo_name'].transform(lambda x: ', '.join(x))


In [27]:
genes_to_disease=genes_to_disease[['text']]
data_=data_[['text']]
df=df[['text']]
df2=df2[['text']]

In [29]:
combined_df = pd.concat([genes_to_disease, data_, df, df2], ignore_index=True)
combined_df.to_csv('finetuning_datasets.csv')